In [26]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

num_classes = 2

data_dir = 'preprocessed'
image_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

base_model = VGG19(
    weights='imagenet',
    include_top=False,
    input_shape=(image_size[0], image_size[1], 3),
)

model = keras.models.Sequential([
    base_model,
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define 5-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

precision_scores, recall_scores, f1_scores = [], [], []

for train_index, val_index in skf.split(train_generator.filenames, train_generator.classes):
    train_data = train_generator
    val_data = validation_generator

    model.fit(train_data, epochs=5)  # Adjust the number of epochs as needed

    y_pred = model.predict(val_data)
    y_true = val_data.classes

    precision = precision_score(y_true, y_pred.argmax(axis=1))
    recall = recall_score(y_true, y_pred.argmax(axis=1))
    f1 = f1_score(y_true, y_pred.argmax(axis=1))

    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

avg_precision = sum(precision_scores) / len(precision_scores)
avg_recall = sum(recall_scores) / len(recall_scores)
avg_f1 = sum(f1_scores) / len(f1_scores)

print(f'Average Precision: {avg_precision}')
print(f'Average Recall: {avg_recall}')
print(f'Average F1 Score: {avg_f1}')


Found 598 images belonging to 2 classes.
Found 148 images belonging to 2 classes.
Epoch 1/5
19/19 [==============================] - 158s 8s/step - loss: 3.9771 - accuracy: 0.5652
Epoch 2/5
19/19 [==============================] - 153s 8s/step - loss: 0.7126 - accuracy: 0.4933
Epoch 3/5
19/19 [==============================] - 151s 8s/step - loss: 0.6907 - accuracy: 0.5452
Epoch 4/5
19/19 [==============================] - 155s 8s/step - loss: 0.6953 - accuracy: 0.5151
Epoch 5/5
5/5 [==============================] - 10s 2s/step
Epoch 1/5
19/19 [==============================] - 151s 8s/step - loss: 0.6919 - accuracy: 0.5318
Epoch 2/5
19/19 [==============================] - 153s 8s/step - loss: 0.6911 - accuracy: 0.5318
Epoch 3/5
19/19 [==============================] - 152s 8s/step - loss: 0.6910 - accuracy: 0.5318
Epoch 4/5
19/19 [==============================] - 153s 8s/step - loss: 0.6913 - accuracy: 0.5318
Epoch 5/5
5/5 [==============================] - 12s 2s/step
Epoch 1/5
19